In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Setting hyperparameters
batch_size = 64
epochs = 15
learning_rate = 0.00001
hidden_shape = 256
num_layer = 3
l1 = False
l2 = False
lr_decay = False

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')


# path = 'drive/MyDrive/code/bitcoin_data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv'
path = "bitcoin_data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv"

In [4]:
df1 = pd.read_csv(path)

# df = df1.dropna(how="any")

print(df1.shape)
df1.isnull().sum()

(4857377, 8)


Timestamp                  0
Open                 1243608
High                 1243608
Low                  1243608
Close                1243608
Volume_(BTC)         1243608
Volume_(Currency)    1243608
Weighted_Price       1243608
dtype: int64

In [5]:
df1['Volume_(BTC)'].fillna(value=0, inplace=True)
df1['Volume_(Currency)'].fillna(value=0, inplace=True)
df1['Weighted_Price'].fillna(value=0, inplace=True)

# next we need to fix the OHLC (open high low close) data which is a continuous timeseries so
# lets fill forwards those values...
df1['Open'].fillna(method='ffill', inplace=True)
df1['High'].fillna(method='ffill', inplace=True)
df1['Low'].fillna(method='ffill', inplace=True)
df1['Close'].fillna(method='ffill', inplace=True)

print(df1.shape)
df1.isnull().sum()

(4857377, 8)


Timestamp            0
Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
dtype: int64

In [6]:
# Generating dataset

class bitcoin(Dataset):
    def __init__(self, X, window):
        self.X = []
        self.y = []
        self.len = X.shape[0]-window-1
        # print(self.len)
        for i in range(0, X.shape[0]-window-1):
          self.X.append(X[i:window+i])
          self.y.append(X[window+i])

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        # print(index)
        # print(self.X[index])
        # print(self.y[index])
        return torch.Tensor(self.X[index]), torch.Tensor([self.y[index]])

In [7]:
scaler = MinMaxScaler()
scaler.fit(df1["Weighted_Price"].to_numpy().reshape(-1, 1))
data = scaler.transform(df1["Weighted_Price"].to_numpy().reshape(-1, 1))

In [8]:
# Sequence lenght is 60.
window = 60

In [9]:
train = bitcoin(data.squeeze()[2400000:4500000], window)

In [10]:
val = bitcoin(data.squeeze()[4500000:4750000], window)

In [11]:
test = bitcoin(data.squeeze()[4750000:4800000], window)

In [12]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

In [13]:
dataset_iter = iter(train_loader)
temp = next(dataset_iter)
features, labels = temp
print(features.shape, labels.shape)

torch.Size([64, 60]) torch.Size([64, 1])


In [14]:
class RNNForecasting(nn.Module):
    def __init__(self, seq_length, input_shape, hidden_shape, output_shape, num_layers):
        super(RNNForecasting, self).__init__()
        self.hidden = hidden_shape
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_shape, hidden_shape, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_shape * seq_length, output_shape)

    def forward(self, x):
        batch_size = x.size(0)
        h = torch.zeros([self.num_layers, batch_size, self.hidden]).cuda()
        c = torch.zeros([self.num_layers, batch_size, self.hidden]).cuda()
        # h = torch.zeros([self.num_layers, batch_size, self.hidden])
        # c = torch.zeros([self.num_layers, batch_size, self.hidden])
        out, _ = self.rnn(x, (h, c))
        out = self.linear(out.contiguous().view(out.shape[0], -1))

        return out

    def compute_l1(self, w):
        return torch.abs(w).sum()

    def compute_l2(self, w):
      return torch.square(w).sum()

In [15]:
input_shape = 1
model = RNNForecasting(window, input_shape, hidden_shape, 1, num_layers=3).cuda()
# model = RNNForecasting(window, input_shape, hidden_shape, 1, num_layers=3)

In [16]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.9)

In [17]:
# Training
l1_weight = 0.01
l2_weight = 0.01
for i in range(epochs):
    mse_train = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        batch_x = batch_x.unsqueeze(2)
        optimizer.zero_grad()
        y_pred = model(batch_x)
        l = criterion(y_pred, batch_y)

    #     # Using l1 regularization
    #     if l1:
    #         l1_parameters = []
    #         for parameter in model.parameters():
    #             l1_parameters.append(parameter.view(-1))
    #         L1 = l1_weight * model.compute_l1(torch.cat(l1_parameters))
    #         l += L1

    #     # Using l2 regularization
    #     if l2:
    #         l2_parameters = []
    #         for parameter in model.parameters():
    #             l2_parameters.append(parameter.view(-1))
    #         L1 = l2_weight * model.compute_l2(torch.cat(l2_parameters))
    #         l += L1

        l.backward()
        mse_train += l.item()*batch_x.shape[0]
        optimizer.step()
    # if lr_decay:
    #     epoch_scheduler.step()
    with torch.no_grad():
        mse_val = 0
        preds = []
        true = []
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            batch_x = batch_x.unsqueeze(2)
            output = model(batch_x)
            preds.append(output.detach().cpu().numpy())
            true.append(batch_y.detach().cpu().numpy())
            # preds.append(output.detach().numpy())
            # true.append(batch_y.detach().numpy())
            mse_val += criterion(output, batch_y).item()*batch_x.shape[0]
    preds = np.concatenate(preds)
    true = np.concatenate(true)

    print("Epoch: ", i, "train: ", (mse_train/train.__len__())**0.5, "val: ", (mse_val/val.__len__())**0.5)

KeyboardInterrupt: 

In [ ]:
preds.shape, true.shape

In [ ]:
preds_reshaped = preds[996:].reshape(-1, 500)
true_reshaped = true[996:].reshape(-1, 500)

preds_avg = np.mean(preds_reshaped, axis=1)
true_avg = np.mean(true_reshaped, axis=1)

plt.figure(figsize=(10, 7))
plt.plot(preds_avg, 'b', label='Preds')
plt.plot(true_avg, 'g', label='True')
plt.legend()
plt.show()

In [ ]:
# Creating/updating config file with hyperparameters and model performance
import os
from pathlib import Path
import json

filepath = "config/config.json"
filepath = Path(filepath)
filedir, filename = os.path.split(filepath)

dic = {
    "model": "LSTM",
    "batch_size": batch_size,
    "epochs": epochs,
    "learning_rate": learning_rate,
    "hidden_shape": hidden_shape,
    "num_layer": num_layer,
    "l1": l1,
    "l2": l2,
    "lr_decay": lr_decay
}

# Serializing json
json_object = json.dumps(dic, indent=4)

if filedir != "":
    os.makedirs(filedir, exist_ok=True)
    print(f"Creating directory:{filedir} for the file {filename}")

    
if (not os.path.exists(filepath)) or (os.path.getsize(filepath) == 0):
    # size!=0 means there is content in the file, so do not replace with write operation.
    with open(filepath,'w') as f:
        f.write(json_object, f, indent=4)
        print("New config file is created with first configuration for the model")

else:
    with open(filepath,'a') as f:
        f.write(json_object, f, indent=4)
        print("New configuration for the model added")